In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import warnings
from scipy import stats
from IPython.core.interactiveshell import InteractiveShell

In [2]:
# Libraries Settings
warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=Warning)
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
np.set_printoptions(suppress=True)

def set_seed(seed=42):
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    
pd.set_option("display.width", 100)
pd.set_option("display.max_columns", 60)
pd.set_option("display.max_rows", 25)
pd.set_option("display.float_format", lambda x: "%.3f" % x)

In [3]:
df_deals = pd.read_excel ('data/Deals (investment).xlsx')
df_deals.head()

1_post_date                    2_post_title 3_amount  4_Stake 5_funding_round  \
0  2008-09-01                            biNu   600000      NaN            Seed   
1  2008-11-01                            biNu   400000      NaN           Angel   
2  2009-12-03                         AllLife  6000000      NaN  Private Equity   
3  2009-12-11  Bridge International Academies  1800000      NaN           Grant   
4  2010-04-01                            biNu   320000      NaN           Angel   

            6_investors                                           7_source  Country/Town  \
0           Artesian VC  https://www.crunchbase.com/search/funding_roun...           NaN   
1           Undisclosed  https://www.crunchbase.com/search/funding_roun...     Australia   
2  LeapFrog Investments  http://www.prnewswire.co.uk/news-releases/leap...  South Africa   
3       Omidyar Network  http://foundationcenter.org/pnd/news/story.jht...           NaN   
4           Undisclosed  https://www.crunchbase.com/search/funding_roun...           NaN   

    Country(HQ)              Category             main_sector Check vs Companies  \
0           NaN                   NaN                     NaN                NaN   
1     Australia       Mobile Internet  Information Technology               biNu   
2  South Africa  Insurance, InsurTech      Financial Services            AllLife   
3           NaN                   NaN                     NaN                NaN   
4           NaN                   NaN                     NaN                NaN   

   Deals Information, Level of Completeness duplicated_conc Duplication  Disclosed   Founded  \
0                                     0.500             NaN         NaN         NaN      NaN   
1                                     0.900             NaN         NaN       1.000 2008.000   
2                                     0.900             NaN         NaN       1.000 2004.000   
3                                     0.643             NaN         NaN       1.000      NaN   
4                                     0.500             NaN         NaN         NaN      NaN   

   Total  Disclosed Funding  DA Classification_African Company (Yes = 0; No = 1)  \
0                       NaN                                                NaN     
1              14220000.000                                              1.000     
2              12700000.000                                              0.000     
3              27800000.000                                                NaN     
4                       NaN                                                NaN     

   funding round, DA                                 Industry, DA     Year  Month  Quarter  Half  \
0                NaN                                          NaN 2008.000  9.000    3.000 2.000   
1            2. Seed  Other Technologies & Information Technology 2008.000 11.000    4.000 2.000   
2  6. Private Equity                           Financial Services 2009.000 12.000    4.000 2.000   
3           1. Grant                                          NaN 2009.000 12.000    4.000 2.000   
4                NaN                                          NaN 2010.000  4.000    2.000 1.000   

   Number of Investors       Investor 1 Investor 2 Investor 3 Investor 4 Investor 5 Investor 6  \
0                1.000            #REF!          0          0          0          0          0   
1                1.000            #REF!          0          0          0          0          0   
2                1.000            #REF!          0          0          0          0          0   
3                1.000  Omidyar Network          0          0          0          0          0   
4                1.000            #REF!          0          0          0          0          0   

  Investor 7 Investor 8 Investor 9 Investor 10 Investor 11 Investor 12 Investor 13 Investor 14  \
0          0          0          0           0           0           0     

In [4]:
def check_df(dataframe):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### Head #####################")
    print(dataframe.head(5))
    print("##################### NA #####################")
    print(dataframe.isnull().sum())
    print("##################### Quantiles #####################")
    print(dataframe.quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

check_df(df_deals)

##################### Shape #####################
(2059, 59)
##################### Types #####################
1_post_date        datetime64[ns]
2_post_title               object
3_amount                   object
4_Stake                   float64
5_funding_round            object
                        ...      
Investor 11.1              object
Investor 12.1              object
Investor 13.1              object
Investor 14.1              object
Investor 15.1             float64
Length: 59, dtype: object
##################### Head #####################
  1_post_date                    2_post_title 3_amount  4_Stake 5_funding_round  \
0  2008-09-01                            biNu   600000      NaN            Seed   
1  2008-11-01                            biNu   400000      NaN           Angel   
2  2009-12-03                         AllLife  6000000      NaN  Private Equity   
3  2009-12-11  Bridge International Academies  1800000      NaN           Grant   
4  2010-04-01            

In [5]:
cols = df_deals.columns

In [6]:
cols

Index(['1_post_date', '2_post_title', '3_amount', '4_Stake', '5_funding_round', '6_investors',
       '7_source', 'Country/Town', 'Country(HQ)', 'Category', 'main_sector', 'Check vs Companies',
       'Deals Information, Level of Completeness', 'duplicated_conc', 'Duplication', 'Disclosed ',
       'Founded', 'Total  Disclosed Funding',
       'DA Classification_African Company (Yes = 0; No = 1)', 'funding round, DA', 'Industry, DA',
       'Year', 'Month', 'Quarter', 'Half', 'Number of Investors', 'Investor 1', 'Investor 2',
       'Investor 3', 'Investor 4', 'Investor 5', 'Investor 6', 'Investor 7', 'Investor 8',
       'Investor 9', 'Investor 10', 'Investor 11', 'Investor 12', 'Investor 13', 'Investor 14',
       'Investor 15', 'Cummulative Deals (Disclosed)', 'Cummulative Deals (Undisclosed)',
       'Cummulative Amount', 'Investor 1.1', 'Investor 2.1', 'Investor 3.1', 'Investor 4.1',
       'Investor 5.1', 'Investor 6.1', 'Investor 7.1', 'Investor 8.1', 'Investor 9.1',
       'Inv

In [7]:
for cat_col in cols:
    print(f"{cat_col}: {df_deals[cat_col].nunique()} uniqueness variable")

1_post_date: 695 uniqueness variable
2_post_title: 800 uniqueness variable
3_amount: 281 uniqueness variable
4_Stake: 7 uniqueness variable
5_funding_round: 27 uniqueness variable
6_investors: 717 uniqueness variable
7_source: 1070 uniqueness variable
Country/Town: 34 uniqueness variable
Country(HQ): 34 uniqueness variable
Category: 660 uniqueness variable
main_sector: 48 uniqueness variable
Check vs Companies: 727 uniqueness variable
Deals Information, Level of Completeness: 6 uniqueness variable
duplicated_conc: 1120 uniqueness variable
Duplication: 1 uniqueness variable
Disclosed : 2 uniqueness variable
Founded: 23 uniqueness variable
Total  Disclosed Funding: 280 uniqueness variable
DA Classification_African Company (Yes = 0; No = 1): 2 uniqueness variable
funding round, DA: 8 uniqueness variable
Industry, DA: 13 uniqueness variable
Year: 14 uniqueness variable
Month: 12 uniqueness variable
Quarter: 4 uniqueness variable
Half: 2 uniqueness variable
Number of Investors: 12 uniquenes

In [8]:
df_deals = df_deals.drop(['Investor 13.1','Investor 14.1','Investor 15.1','Investor 15','Duplication'], axis=1)

In [9]:
df_deals.shape

(2059, 54)

In [10]:
df_deals.head(20)

1_post_date                    2_post_title     3_amount  4_Stake 5_funding_round  \
0   2008-09-01                            biNu       600000      NaN            Seed   
1   2008-11-01                            biNu       400000      NaN           Angel   
2   2009-12-03                         AllLife      6000000      NaN  Private Equity   
3   2009-12-11  Bridge International Academies      1800000      NaN           Grant   
4   2010-04-01                            biNu       320000      NaN           Angel   
5   2010-08-20                       Kopo Kopo        75000      NaN            Seed   
6   2010-09-30                            Paga       700000      NaN            Seed   
7   2010-12-01                          Wuzzuf       150000      NaN           Grant   
8   2011-05-01      African Leadership Academy      1500000      NaN           Grant   
9   2011-10-01                          M-KOPA  Undisclosed      NaN        Series A   
10  2011-10-01                       Cellulant      1500000      NaN        Series A   
11  2011-11-17                       Kopo Kopo       865000      NaN            Seed   
12  2011-11-17                       Kopo Kopo       864900      NaN            Seed   
13  2011-12-01                         iROKOtv      3000000      NaN        Series A   
14  2012-01-01                     Jumia Group     45000000      NaN        Series A   
15  2012-01-01                           Konga      3500000      NaN            Seed   
16  2012-01-02  Bridge International Academies  Undisclosed      NaN        Series C   
17  2012-02-01                         elmenus  Undisclosed      NaN           Angel   
18  2012-02-29                           Zoona      4000000      NaN        Series A   
19  2012-02-29                           Zoona       500000      NaN  Debt Financing   

                                          6_investors  \
0                                         Artesian VC   
1                                         Undisclosed   
2                                LeapFrog Investments   
3                                     Omidyar Network   
4                                         Undisclosed   
5                                First Light Ventures   
6                                          Tim Draper   
7                                         Undisclosed   
8                                     Omidyar Network   
9                                         Undisclosed   
10                                    TBL Mirror Fund   
11                                        Undisclosed   
12                                        Undisclosed   
13                            Tiger Global Management   
14  Blakeney Management, Rocket Internet, Millicom...   
15                                        Kinnevik AB   
16  Khosla Impact, Khosla Ventures, Learn Capital,...   
17                                        Undisclosed   
18                            Accion, Omidyar Network   
19          Mennonite Economic Development Associates   

                                             7_source  Country/Town   Country(HQ)  \
0   https://www.crunchbase.com/search/funding_roun...           NaN           NaN   
1   https://www.crunchbase.com/search/funding_roun...     Australia     Australia   
2   http://www.prnewswire.co.uk/news-releases/leap...  South Africa  South Africa   
3   http://foundationcenter.org/pnd/news/story.jht...           NaN           NaN   
4   https://www.crunchbase.com/search/funding_roun...           NaN           NaN   
5   https://www.crunchbase.com/organization/kopo-k...         Kenya         Kenya   
6   https://www.crunchbase.com/funding_round/paga-...       Nigeria       Nigeria   
7   https://www.crunchbase.com/funding_round/wuzzu...         Egypt         Egypt   
8   https://www.crunchbase.com/funding_round/afric...  South Africa  South Africa   
9   https://www.crunchbase.com/funding_round/m-kop...         Kenya         Kenya   
10  https://www.busin

In [11]:
#find duplicate rows across all columns
duplicateRows = df_deals[df_deals.duplicated()]

In [12]:
duplicateRows

1_post_date 2_post_title 3_amount  4_Stake 5_funding_round  \
489   2018-04-19     Shezlong   350000      NaN            Seed   
558   2018-06-05       Onesha    50000      NaN            Seed   
1279         NaT          NaN      NaN      NaN             NaN   
1280         NaT          NaN      NaN      NaN             NaN   
1281         NaT          NaN      NaN      NaN             NaN   
...          ...          ...      ...      ...             ...   
2054         NaT          NaN      NaN      NaN             NaN   
2055         NaT          NaN      NaN      NaN             NaN   
2056         NaT          NaN      NaN      NaN             NaN   
2057         NaT          NaN      NaN      NaN             NaN   
2058         NaT          NaN      NaN      NaN             NaN   

                                 6_investors                                           7_source  \
489   500 Startups, Endure Capital, HIMangel           https://www.menabytes.com/shezlong-350k/   
558                       Pangea Accelerator  http://disrupt-africa.com/2018/06/pangea-inves...   
1279                                     NaN                                                NaN   
1280                                     NaN                                                NaN   
1281                                     NaN                                                NaN   
...                                      ...                                                ...   
2054                                     NaN                                                NaN   
2055                                     NaN                                                NaN   
2056                                     NaN                                                NaN   
2057                                     NaN                                                NaN   
2058                                     NaN                                                NaN   

     Country/Town Country(HQ)                                           Category  \
489         Egypt       Egypt                      Health Care, Medical, mHealth   
558         Kenya       Kenya  Architecture, Brand Marketing, Digital Marketi...   
1279          NaN         NaN                                                NaN   
1280          NaN         NaN                                                NaN   
1281          NaN         NaN                                                NaN   
...           ...         ...                                                ...   
2054          NaN         NaN                                                NaN   
2055          NaN         NaN                                                NaN   
2056          NaN         NaN                                                NaN   
2057          NaN         NaN                                                NaN   
2058          NaN         NaN                                                NaN   

                main_sector Check vs Companies  Deals Information, Level of Completeness  \
489     Healthcare & Pharma           Shezlong                                     0.900   
558   Media & Entertainment             Onesha                                     0.900   
1279                    NaN                NaN                                       NaN   
1280                    NaN                NaN                                       NaN   
1281                    NaN                NaN                                       NaN   
...                     ...                ...                                       ...   
2054                    NaN                NaN                                       NaN   
2055                    NaN                NaN                                       NaN   
2056                    NaN                NaN                                       NaN   
2057                    NaN                NaN                                    

In [13]:
df_deals

1_post_date                    2_post_title 3_amount  4_Stake 5_funding_round  \
0     2008-09-01                            biNu   600000      NaN            Seed   
1     2008-11-01                            biNu   400000      NaN           Angel   
2     2009-12-03                         AllLife  6000000      NaN  Private Equity   
3     2009-12-11  Bridge International Academies  1800000      NaN           Grant   
4     2010-04-01                            biNu   320000      NaN           Angel   
...          ...                             ...      ...      ...             ...   
2054         NaT                             NaN      NaN      NaN             NaN   
2055         NaT                             NaN      NaN      NaN             NaN   
2056         NaT                             NaN      NaN      NaN             NaN   
2057         NaT                             NaN      NaN      NaN             NaN   
2058         NaT                             NaN      NaN      NaN             NaN   

               6_investors                                           7_source  Country/Town  \
0              Artesian VC  https://www.crunchbase.com/search/funding_roun...           NaN   
1              Undisclosed  https://www.crunchbase.com/search/funding_roun...     Australia   
2     LeapFrog Investments  http://www.prnewswire.co.uk/news-releases/leap...  South Africa   
3          Omidyar Network  http://foundationcenter.org/pnd/news/story.jht...           NaN   
4              Undisclosed  https://www.crunchbase.com/search/funding_roun...           NaN   
...                    ...                                                ...           ...   
2054                   NaN                                                NaN           NaN   
2055                   NaN                                                NaN           NaN   
2056                   NaN                                                NaN           NaN   
2057                   NaN                                                NaN           NaN   
2058                   NaN                                                NaN           NaN   

       Country(HQ)              Category             main_sector Check vs Companies  \
0              NaN                   NaN                     NaN                NaN   
1        Australia       Mobile Internet  Information Technology               biNu   
2     South Africa  Insurance, InsurTech      Financial Services            AllLife   
3              NaN                   NaN                     NaN                NaN   
4              NaN                   NaN                     NaN                NaN   
...            ...                   ...                     ...                ...   
2054           NaN                   NaN                     NaN                NaN   
2055           NaN                   NaN                     NaN                NaN   
2056           NaN                   NaN                     NaN                NaN   
2057           NaN                   NaN                     NaN                NaN   
2058           NaN                   NaN                     NaN                NaN   

      Deals Information, Level of Completeness duplicated_conc  Disclosed   Founded  \
0                                        0.500             NaN         NaN      NaN   
1                                        0.900             NaN       1.000 2008.000   
2                                        0.900             NaN       1.000 2004.000   
3                                        0.643             NaN       1.000      NaN   
4                                        0.500             NaN         NaN      NaN   
...                                        ...             ...         ...      ...   
2054                                       NaN             NaN         NaN      NaN   
2055                                       NaN             NaN         NaN      NaN   
205

### Data cleaning

###### amount

In [14]:
df_deals['3_amount'].isna().sum()
df_deals['3_amount']

781

0        600000
1        400000
2       6000000
3       1800000
4        320000
         ...   
2054        NaN
2055        NaN
2056        NaN
2057        NaN
2058        NaN
Name: 3_amount, Length: 2059, dtype: object

In [15]:
# Filling all nan values with zero
df_deals['3_amount'] = df_deals['3_amount'].fillna(0)

In [16]:
# df_deals['3_amount'].astype('int')
df_deals['3_amount'] = df_deals['3_amount'].replace('Undisclosed', '0')
df_deals['3_amount'] = df_deals['3_amount'].astype(int)

In [17]:
df_deals['3_amount'].astype('int')
# df_deals['3_amount'].unique()

0        600000
1        400000
2       6000000
3       1800000
4        320000
         ...   
2054          0
2055          0
2056          0
2057          0
2058          0
Name: 3_amount, Length: 2059, dtype: int64

In [18]:
print(df_deals.dtypes)

1_post_date        datetime64[ns]
2_post_title               object
3_amount                    int64
4_Stake                   float64
5_funding_round            object
                        ...      
Investor 8.1               object
Investor 9.1               object
Investor 10.1              object
Investor 11.1              object
Investor 12.1              object
Length: 54, dtype: object


In [19]:
# Unique_deals

###### date

In [20]:
df_deals['1_post_date'] = df_deals['1_post_date'].astype('datetime64')

In [21]:
#filling nan values with mode of the year column
df_deals['1_post_date'] = df_deals['1_post_date'].fillna(df_deals['1_post_date'].mode()[0])

In [22]:
#Add year column from 1_post_date
def get_year(dt):
    return dt.year

df_deals['year'] = df_deals['1_post_date'].map(get_year)
#df_deals

In [23]:
#drop duplicate rows unique deals
Unique_deals_df = df_deals.drop_duplicates()

In [24]:
year_with_highest_deals_df = Unique_deals_df[['year','2_post_title']]

In [25]:
yrVposts = Unique_deals_df.year.value_counts()
yrVposts

2018    510
2019    366
2017    153
2016    108
2015     60
2014     31
2013     20
2012     16
2011      6
2010      4
2008      2
2009      2
2007      1
Name: year, dtype: int64

In [26]:
yrVposts.to_csv('yearVposts.json',index = False)

value of investment yearly, monthly quartly

In [27]:
Unique_deals_df

1_post_date                    2_post_title  3_amount  4_Stake 5_funding_round  \
0     2008-09-01                            biNu    600000      NaN            Seed   
1     2008-11-01                            biNu    400000      NaN           Angel   
2     2009-12-03                         AllLife   6000000      NaN  Private Equity   
3     2009-12-11  Bridge International Academies   1800000      NaN           Grant   
4     2010-04-01                            biNu    320000      NaN           Angel   
...          ...                             ...       ...      ...             ...   
1276  2019-09-18               Sesi Technologies     50000      NaN           Grant   
1277  2019-09-18                    Maungo Craft     50000      NaN           Grant   
1278  2019-05-03                             NaN         0      NaN             NaN   
1304  2019-05-03                             NaN         0      NaN             NaN   
1533  2019-05-03                             NaN         0      NaN             NaN   

                     6_investors                                           7_source  Country/Town  \
0                    Artesian VC  https://www.crunchbase.com/search/funding_roun...           NaN   
1                    Undisclosed  https://www.crunchbase.com/search/funding_roun...     Australia   
2           LeapFrog Investments  http://www.prnewswire.co.uk/news-releases/leap...  South Africa   
3                Omidyar Network  http://foundationcenter.org/pnd/news/story.jht...           NaN   
4                    Undisclosed  https://www.crunchbase.com/search/funding_roun...           NaN   
...                          ...                                                ...           ...   
1276  GoGettaz Agripreneur Prize  https://disrupt-africa.com/2019/09/botswanan-g...           NaN   
1277  GoGettaz Agripreneur Prize  https://disrupt-africa.com/2019/09/botswanan-g...           NaN   
1278                         NaN                                                NaN           NaN   
1304                         NaN                                                NaN           NaN   
1533                         NaN                                                NaN           NaN   

       Country(HQ)              Category             main_sector Check vs Companies  \
0              NaN                   NaN                     NaN                NaN   
1        Australia       Mobile Internet  Information Technology               biNu   
2     South Africa  Insurance, InsurTech      Financial Services            AllLife   
3              NaN                   NaN                     NaN                NaN   
4              NaN                   NaN                     NaN                NaN   
...            ...                   ...                     ...                ...   
1276           NaN                   NaN                     NaN                NaN   
1277           NaN                   NaN                     NaN                NaN   
1278           NaN                   NaN                     NaN                NaN   
1304           NaN                   NaN                     NaN                NaN   
1533           NaN                   NaN                     NaN                NaN   

      Deals Information, Level of Completeness duplicated_conc  Disclosed   Founded  \
0                                        0.500             NaN         NaN      NaN   
1                                        0.900             NaN       1.000 2008.000   
2                                        0.900             NaN       1.000 2004.000   
3                                        0.643             NaN       1.000      NaN   
4                                        0.500             NaN         NaN      NaN   
...                                        ...             ...         ...      ...   
1276                                       NaN             NaN         NaN      NaN   
1277  

In [28]:
Unique_deals_df['year']

0       2008
1       2008
2       2009
3       2009
4       2010
        ... 
1276    2019
1277    2019
1278    2019
1304    2019
1533    2019
Name: year, Length: 1279, dtype: int64

In [29]:
print(Unique_deals_df.dtypes)
Unique_deals_df['3_amount'].astype(int)

1_post_date        datetime64[ns]
2_post_title               object
3_amount                    int64
4_Stake                   float64
5_funding_round            object
                        ...      
Investor 9.1               object
Investor 10.1              object
Investor 11.1              object
Investor 12.1              object
year                        int64
Length: 55, dtype: object


0        600000
1        400000
2       6000000
3       1800000
4        320000
         ...   
1276      50000
1277      50000
1278          0
1304          0
1533          0
Name: 3_amount, Length: 1279, dtype: int64

In [1]:
#function to shorten amount values
shorten = lambda x: str(round(x/1000, 1)) + 'K'

In [30]:
Unique_deals_df['year'].unique()

array([2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018,
       2019, 2007])

In [31]:
value_of_investment_yearly = Unique_deals_df.groupby('year')['3_amount'].sum()

In [32]:
value_of_investment_yearly

year
2007       3700000
2008       1000000
2009       7800000
2010       1245000
2011       7729900
2012      91930000
2013     194949000
2014     378706400
2015     263932000
2016     717231200
2017     455651700
2018    1035045845
2019    2121503060
Name: 3_amount, dtype: int64

In [33]:
# value_of_investment_yearly.plot(kind='bar')

In [34]:
value_of_investment_quartly = Unique_deals_df.groupby('Quarter')['3_amount'].sum()

In [35]:
value_of_investment_quartly

Quarter
1.000    2778053693
2.000     924334053
3.000     571775146
4.000    1004841213
Name: 3_amount, dtype: int64

In [36]:
value_of_investment_monthly = Unique_deals_df.groupby('Month')['3_amount'].sum()

In [37]:
value_of_investment_monthly

Month
1.000      882073460
2.000     1271514400
3.000      624465833
4.000      345455400
5.000      321937250
6.000      256941403
7.000      167399800
8.000      217561456
9.000      186813890
10.000     212534500
11.000     422839300
12.000     369467413
Name: 3_amount, dtype: int64

In [38]:
value_of_investment_by_sector = Unique_deals_df.groupby('main_sector')['3_amount'].sum()

In [39]:
value_of_investment_by_sector

main_sector
0                                             10260000
Agriculture                                   23126990
Automotive                                     5905400
Commercial & Professional Services           199444480
E-Commerce & Retail                         1293944032
                                               ...    
Transport & Logistics                        194361903
Travel & Leisure                             104508000
Travel and Leisure                              320000
Utilities and Telcommunication Services              0
Utilities and Telecommunication Services       8100000
Name: 3_amount, Length: 48, dtype: int64

In [40]:
value_of_investment_by_level_of_completeness = Unique_deals_df.groupby('Deals Information, Level of Completeness')['3_amount'].sum()

In [41]:
value_of_investment_by_level_of_completeness

Deals Information, Level of Completeness
0.400             0
0.500      12420000
0.643    1160238000
0.800       6700000
0.900    3826138005
1.000        178000
Name: 3_amount, dtype: int64

In [42]:
value_of_investment_by_country = Unique_deals_df.groupby('Country/Town')['3_amount'].sum()

In [50]:
value_of_investment_by_country

Country/Town
0                            16100000
Algeria                       1700000
Angola                         689000
Australia                     3400000
Benin                               0
                              ...    
United Kingdom              382411500
United States               204300000
United States of America       697000
Zambia                       23536000
Zimbabwe                     36625000
Name: 3_amount, Length: 34, dtype: int64

In [58]:
#shortening amount values
amount = value_of_investment_by_country.apply(shorten)

In [59]:
amount

Country/Town
0                            16100.0K
Algeria                       1700.0K
Angola                         689.0K
Australia                     3400.0K
Benin                            0.0K
                              ...    
United Kingdom              382411.5K
United States               204300.0K
United States of America       697.0K
Zambia                       23536.0K
Zimbabwe                     36625.0K
Name: 3_amount, Length: 34, dtype: object

In [60]:
amount.to_csv('value_of_investment_by_country.csv', index=True)

##### Deals list 

In [61]:
Unique_deals_df

1_post_date                    2_post_title  3_amount  4_Stake 5_funding_round  \
0     2008-09-01                            biNu    600000      NaN            Seed   
1     2008-11-01                            biNu    400000      NaN           Angel   
2     2009-12-03                         AllLife   6000000      NaN  Private Equity   
3     2009-12-11  Bridge International Academies   1800000      NaN           Grant   
4     2010-04-01                            biNu    320000      NaN           Angel   
...          ...                             ...       ...      ...             ...   
1276  2019-09-18               Sesi Technologies     50000      NaN           Grant   
1277  2019-09-18                    Maungo Craft     50000      NaN           Grant   
1278  2019-05-03                             NaN         0      NaN             NaN   
1304  2019-05-03                             NaN         0      NaN             NaN   
1533  2019-05-03                             NaN         0      NaN             NaN   

                     6_investors                                           7_source  Country/Town  \
0                    Artesian VC  https://www.crunchbase.com/search/funding_roun...           NaN   
1                    Undisclosed  https://www.crunchbase.com/search/funding_roun...     Australia   
2           LeapFrog Investments  http://www.prnewswire.co.uk/news-releases/leap...  South Africa   
3                Omidyar Network  http://foundationcenter.org/pnd/news/story.jht...           NaN   
4                    Undisclosed  https://www.crunchbase.com/search/funding_roun...           NaN   
...                          ...                                                ...           ...   
1276  GoGettaz Agripreneur Prize  https://disrupt-africa.com/2019/09/botswanan-g...           NaN   
1277  GoGettaz Agripreneur Prize  https://disrupt-africa.com/2019/09/botswanan-g...           NaN   
1278                         NaN                                                NaN           NaN   
1304                         NaN                                                NaN           NaN   
1533                         NaN                                                NaN           NaN   

       Country(HQ)              Category             main_sector Check vs Companies  \
0              NaN                   NaN                     NaN                NaN   
1        Australia       Mobile Internet  Information Technology               biNu   
2     South Africa  Insurance, InsurTech      Financial Services            AllLife   
3              NaN                   NaN                     NaN                NaN   
4              NaN                   NaN                     NaN                NaN   
...            ...                   ...                     ...                ...   
1276           NaN                   NaN                     NaN                NaN   
1277           NaN                   NaN                     NaN                NaN   
1278           NaN                   NaN                     NaN                NaN   
1304           NaN                   NaN                     NaN                NaN   
1533           NaN                   NaN                     NaN                NaN   

      Deals Information, Level of Completeness duplicated_conc  Disclosed   Founded  \
0                                        0.500             NaN         NaN      NaN   
1                                        0.900             NaN       1.000 2008.000   
2                                        0.900             NaN       1.000 2004.000   
3                                        0.643             NaN       1.000      NaN   
4                                        0.500             NaN         NaN      NaN   
...                                        ...             ...         ...      ...   
1276                                       NaN             NaN         NaN      NaN   
1277  

In [62]:
Unique_deals_df.columns

Index(['1_post_date', '2_post_title', '3_amount', '4_Stake', '5_funding_round', '6_investors',
       '7_source', 'Country/Town', 'Country(HQ)', 'Category', 'main_sector', 'Check vs Companies',
       'Deals Information, Level of Completeness', 'duplicated_conc', 'Disclosed ', 'Founded',
       'Total  Disclosed Funding', 'DA Classification_African Company (Yes = 0; No = 1)',
       'funding round, DA', 'Industry, DA', 'Year', 'Month', 'Quarter', 'Half',
       'Number of Investors', 'Investor 1', 'Investor 2', 'Investor 3', 'Investor 4', 'Investor 5',
       'Investor 6', 'Investor 7', 'Investor 8', 'Investor 9', 'Investor 10', 'Investor 11',
       'Investor 12', 'Investor 13', 'Investor 14', 'Cummulative Deals (Disclosed)',
       'Cummulative Deals (Undisclosed)', 'Cummulative Amount', 'Investor 1.1', 'Investor 2.1',
       'Investor 3.1', 'Investor 4.1', 'Investor 5.1', 'Investor 6.1', 'Investor 7.1',
       'Investor 8.1', 'Investor 9.1', 'Investor 10.1', 'Investor 11.1', 'Invest

In [63]:
deals_list_df = Unique_deals_df.drop(['7_source','4_Stake','Deals Information, Level of Completeness', 'duplicated_conc', 'Disclosed ', 'Founded',
       'Total  Disclosed Funding', 'DA Classification_African Company (Yes = 0; No = 1)',
       'funding round, DA', 'Industry, DA', 'Year', 'Month', 'Quarter', 'Half',
       'Number of Investors', 'Investor 1', 'Investor 2', 'Investor 3', 'Investor 4', 'Investor 5',
       'Investor 6', 'Investor 7', 'Investor 8', 'Investor 9', 'Investor 10', 'Investor 11',
       'Investor 12', 'Investor 13', 'Investor 14', 'Cummulative Deals (Disclosed)',
       'Cummulative Deals (Undisclosed)', 'Cummulative Amount', 'Investor 1.1', 'Investor 2.1',
       'Investor 3.1', 'Investor 4.1', 'Investor 5.1', 'Investor 6.1', 'Investor 7.1',
       'Investor 8.1', 'Investor 9.1', 'Investor 10.1', 'Investor 11.1', 'Investor 12.1', 'year'], axis=1)

In [64]:
deals_list_df

1_post_date                    2_post_title  3_amount 5_funding_round  \
0     2008-09-01                            biNu    600000            Seed   
1     2008-11-01                            biNu    400000           Angel   
2     2009-12-03                         AllLife   6000000  Private Equity   
3     2009-12-11  Bridge International Academies   1800000           Grant   
4     2010-04-01                            biNu    320000           Angel   
...          ...                             ...       ...             ...   
1276  2019-09-18               Sesi Technologies     50000           Grant   
1277  2019-09-18                    Maungo Craft     50000           Grant   
1278  2019-05-03                             NaN         0             NaN   
1304  2019-05-03                             NaN         0             NaN   
1533  2019-05-03                             NaN         0             NaN   

                     6_investors  Country/Town   Country(HQ)              Category  \
0                    Artesian VC           NaN           NaN                   NaN   
1                    Undisclosed     Australia     Australia       Mobile Internet   
2           LeapFrog Investments  South Africa  South Africa  Insurance, InsurTech   
3                Omidyar Network           NaN           NaN                   NaN   
4                    Undisclosed           NaN           NaN                   NaN   
...                          ...           ...           ...                   ...   
1276  GoGettaz Agripreneur Prize           NaN           NaN                   NaN   
1277  GoGettaz Agripreneur Prize           NaN           NaN                   NaN   
1278                         NaN           NaN           NaN                   NaN   
1304                         NaN           NaN           NaN                   NaN   
1533                         NaN           NaN           NaN                   NaN   

                 main_sector Check vs Companies  
0                        NaN                NaN  
1     Information Technology               biNu  
2         Financial Services            AllLife  
3                        NaN                NaN  
4                        NaN                NaN  
...                      ...                ...  
1276                     NaN                NaN  
1277                     NaN                NaN  
1278                     NaN                NaN  
1304                     NaN                NaN  
1533                     NaN                NaN  

[1279 rows x 10 columns]

In [65]:
plt.figure(figsize=(20,5))
sns.heatmap(deals_list_df.isnull(),yticklabels=False,cbar=False,cmap='viridis')
plt.plot

<Figure size 1440x360 with 0 Axes>

<AxesSubplot:>

<function matplotlib.pyplot.plot(*args, scalex=True, scaley=True, data=None, **kwargs)>

In [66]:
deals_list_df['2_post_title'].isna().sum()

3

In [68]:
# deals_list_df['2_post_title'].unique()

In [71]:
deals_list_df['2_post_title'].dropna()

0                                 biNu
1                                 biNu
2                              AllLife
3       Bridge International Academies
4                                 biNu
                     ...              
1273                         Career180
1274                        Yebo Fresh
1275                            Dabchy
1276                 Sesi Technologies
1277                      Maungo Craft
Name: 2_post_title, Length: 1276, dtype: object

In [72]:
deals_list_df['2_post_title'].isna().sum()

3

In [81]:
ws_df =deals_list_df.dropna() 

In [82]:
ws_df

1_post_date       2_post_title  3_amount  5_funding_round           6_investors  \
1     2008-11-01               biNu    400000            Angel           Undisclosed   
2     2009-12-03            AllLife   6000000   Private Equity  LeapFrog Investments   
5     2010-08-20          Kopo Kopo     75000             Seed  First Light Ventures   
6     2010-09-30               Paga    700000             Seed            Tim Draper   
7     2010-12-01             Wuzzuf    150000            Grant           Undisclosed   
...          ...                ...       ...              ...                   ...   
1246  2019-04-01        Jumia Group  56000000  Corporate Round            Mastercard   
1247  2018-12-15              MPost    100000             Seed       Takumo Terakubo   
1248  2019-09-02   Livestock Wealth    131000   Debt Financing             AlphaCode   
1249  2019-09-02       Zande Africa    654000   Debt Financing             AlphaCode   
1250  2019-09-02  Bright On Capital    654000   Debt Financing             AlphaCode   

      Country/Town   Country(HQ)                                           Category  \
1        Australia     Australia                                    Mobile Internet   
2     South Africa  South Africa                               Insurance, InsurTech   
5            Kenya         Kenya  Enterprise Software, Financial Services, Infor...   
6          Nigeria       Nigeria  Money Transfer, Airtime, Bill Payments, and SM...   
7            Egypt         Egypt  Information Services, Information Technology, ...   
...            ...           ...                                                ...   
1246       Nigeria       Nigeria              E-Commerce, Internet, Shopping Online   
1247         Kenya         Kenya                                Mobile, Mobile Apps   
1248  South Africa  South Africa                          Agriculture, Crowdfunding   
1249  South Africa  South Africa                      Loan financing, SME financing   
1250  South Africa  South Africa                               Peer-to-peer lending   

                 main_sector Check vs Companies  
1     Information Technology               biNu  
2         Financial Services            AllLife  
5         Financial Services          Kopo Kopo  
6         Financial Services               Paga  
7                Recruitment             Wuzzuf  
...                      ...                ...  
1246     E-Commerce & Retail        Jumia Group  
1247  Information Technology              MPost  
1248      Financial Services   Livestock Wealth  
1249      Financial Services       Zande Africa  
1250      Financial Services  Bright On Capital  

[1174 rows x 10 columns]

In [83]:
ws_df = ws_df.drop(['Check vs Companies'], axis=1)

In [84]:
#Renaming column names
ws_df.rename(
    columns={"1_post_date": "DATE", "2_post_title": "COMPANYNAME", "3_amount": "AMOUNT", "5_funding_round": "ROUND", "6_investors": "INVESTORS", "Country/Town": "COUNTRY", "Country(HQ)": "COUNTRY(HQ)", "Category": "CATEGORY", "main_sector": "SECTOR"},
    inplace=True,
)

ws

DATE        COMPANYNAME    AMOUNT            ROUND             INVESTORS       COUNTRY  \
1    2008-11-01               biNu    400000            Angel           Undisclosed     Australia   
2    2009-12-03            AllLife   6000000   Private Equity  LeapFrog Investments  South Africa   
5    2010-08-20          Kopo Kopo     75000             Seed  First Light Ventures         Kenya   
6    2010-09-30               Paga    700000             Seed            Tim Draper       Nigeria   
7    2010-12-01             Wuzzuf    150000            Grant           Undisclosed         Egypt   
...         ...                ...       ...              ...                   ...           ...   
1246 2019-04-01        Jumia Group  56000000  Corporate Round            Mastercard       Nigeria   
1247 2018-12-15              MPost    100000             Seed       Takumo Terakubo         Kenya   
1248 2019-09-02   Livestock Wealth    131000   Debt Financing             AlphaCode  South Africa   
1249 2019-09-02       Zande Africa    654000   Debt Financing             AlphaCode  South Africa   
1250 2019-09-02  Bright On Capital    654000   Debt Financing             AlphaCode  South Africa   

       COUNTRY(HQ)                                           CATEGORY                  SECTOR  
1        Australia                                    Mobile Internet  Information Technology  
2     South Africa                               Insurance, InsurTech      Financial Services  
5            Kenya  Enterprise Software, Financial Services, Infor...      Financial Services  
6          Nigeria  Money Transfer, Airtime, Bill Payments, and SM...      Financial Services  
7            Egypt  Information Services, Information Technology, ...             Recruitment  
...            ...                                                ...                     ...  
1246       Nigeria              E-Commerce, Internet, Shopping Online     E-Commerce & Retail  
1247         Kenya                                Mobile, Mobile Apps  Information Technology  
1248  South Africa                          Agriculture, Crowdfunding      Financial Services  
1249  South Africa                      Loan financing, SME financing      Financial Services  
1250  South Africa                               Peer-to-peer lending      Financial Services  

[1174 rows x 9 columns]

In [80]:
#function to shorten amount values, by reprenting them in millions
shorten = lambda x: str(round(x/1000000, 1)) + 'M'

In [86]:
ws_df['AMOUNT'] = ws_df['AMOUNT'].apply(shorten)

In [87]:
ws_df.head()

DATE COMPANYNAME AMOUNT           ROUND             INVESTORS       COUNTRY   COUNTRY(HQ)  \
1 2008-11-01        biNu   0.4M           Angel           Undisclosed     Australia     Australia   
2 2009-12-03     AllLife   6.0M  Private Equity  LeapFrog Investments  South Africa  South Africa   
5 2010-08-20   Kopo Kopo   0.1M            Seed  First Light Ventures         Kenya         Kenya   
6 2010-09-30        Paga   0.7M            Seed            Tim Draper       Nigeria       Nigeria   
7 2010-12-01      Wuzzuf   0.1M           Grant           Undisclosed         Egypt         Egypt   

                                            CATEGORY                  SECTOR  
1                                    Mobile Internet  Information Technology  
2                               Insurance, InsurTech      Financial Services  
5  Enterprise Software, Financial Services, Infor...      Financial Services  
6  Money Transfer, Airtime, Bill Payments, and SM...      Financial Services  
7  Information Services, Information Technology, ...             Recruitment

In [88]:
ws_df.to_csv('deals_list.csv', index=True)